Based on https://ymd_h.gitlab.io/ymd_blog/posts/gym_on_google_colab_with_gnwrapper/

## Setup

In [ ]:
!sudo apt-get install -y xvfb ffmpeg x11-utils
!pip install -q 'gym==0.10.11'
!pip install -q 'imageio==2.4.0'
!pip install -q PILLOW
!pip install -q 'pyglet==1.3.2'
!pip install -q pyvirtualdisplay
!pip install -q tf-agents
!pip install colabgymrender
!pip install gym_sokoban
!pip install gym
!apt-get install python-opengl -y
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install piglet

In [ ]:

import gym
import gym_sokoban

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

from colabgymrender.recorder import Recorder
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
Display().start()
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay


import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [ ]:
env = gym.make("PushAndPull-Sokoban-v2")
screen = env.render(mode='rgb_array')
print(screen.shape)


In [ ]:
screen.shape

In [ ]:
plt.imshow(screen)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from skimage import color
# img = color.rgb2gray(screen)
# plt.imshow(img)
screen.shape

In [ ]:
plt.imshow(screen)

In [ ]:
env = gym.make("PushAndPull-Sokoban-v2")

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
class DQN(nn.Module):

    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
        def conv2d_size_out(size, kernel_size = 5, stride = 2):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        self.head = nn.Linear(linear_input_size, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

In [ ]:
resize = T.Compose([
                    T.ToPILImage(),
                    T.Grayscale(num_output_channels=1),
                    T.Resize(80, interpolation=Image.CUBIC),
                    T.ToTensor()])



def get_screen():
    screen = env.render(mode='rgb_array')#.transpose((2, 0, 1))
    #screen = color.rgb2gray(screen)
    return resize(screen)
    

In [ ]:
BATCH_SIZE = 32
GAMMA = 0.9
EPS_START = 1.0
EPS_END = 0.1
EPS_DECAY = 60000
TARGET_UPDATE = 10

# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
init_screen = get_screen()

_, screen_height, screen_width = init_screen.shape

# Get number of actions from gym action space
n_actions = env.action_space.n

policy_net = DQN(screen_height, screen_width, n_actions).to(device)
target_net = DQN(screen_height, screen_width, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

#optimizer = optim.RMSprop(policy_net.parameters())
optimizer = optim.Adam(policy_net.parameters(), lr=0.001)
#torch.nn.SmoothL1Loss(size_average=None, reduce=None, reduction='mean', beta=1.0)


#torch.nn.HuberLoss(reduction='mean', delta=1.0)


In [ ]:
criterion = nn.SmoothL1Loss(beta = 0.1)

In [ ]:

# steps_done += 10
# # # # EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY), steps_done, random.random()
# # # state = get_screen().unsqueeze(0)
# # # state.shape

# # # select_action(state), policy_net(state).max(1)[1].view(1, 1)

# EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY), random.random()

# EPS_DECAY2 = 60000
prev = 1.0

for i in range(100000):
  eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * i / EPS_DECAY)

  if prev - eps_threshold >= 0.1:
    print((i, eps_threshold))
    prev = eps_threshold

In [ ]:
# select_action(state)
# vals = policy_net(state).detach().cpu().numpy().flatten()
# min_vale = np.min(vals)
# min_vale, vals, vals + -min_vale + 0.0000001

In [ ]:
def select_action(state):
    # global steps_done
    # sample = random.random()
    # eps_threshold = EPS_END + (EPS_START - EPS_END) * \
    #     math.exp(-1. * steps_done / EPS_DECAY)
    # steps_done += 1
    # if sample > eps_threshold:
    #     with torch.no_grad():
    #         # t.max(1) will return largest column value of each row.
    #         # second column on max result is index of where max element was
    #         # found, so we pick action with the larger expected reward.
    #         return policy_net(state).max(1)[1].view(1, 1)
    # else:
    #     return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

    vals = policy_net(state).detach().cpu().numpy().flatten()
    min_vale = np.min(vals)
    add = -min_vale + 0.0000001
    if min_vale < 0:
      vals += add

    probs = vals/np.sum(vals)

    action = np.random.choice(n_actions, 1, p=probs)[0]

    return torch.tensor([[action]], device=device, dtype=torch.long)





def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())

In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)

    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    #print(state_action_values - expected_state_action_values.unsqueeze(1))
    

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [ ]:
num_episodes = 10000
episode_durations = []
steps_done = 0
memory = ReplayMemory(2500)
policy_net.train()
target_net.eval()
eps_to_actions = {}
for i_episode in range(num_episodes):
    episoe_states = []
    eps_to_actions[i_episode] = []
    #print(i_episode)
    # Initialize the environment and state
    env.reset()
    #last_screen = get_screen().unsqueeze(0)
    state = get_screen().unsqueeze(0)
    # state = current_screen - last_screen
    for t in count():
        # Select and perform an action
        action = select_action(state)
        
        eps_to_actions[i_episode].append(action)
        _, reward, done, _ = env.step(action.item())
        episoe_states.append((state, reward))
        reward = torch.tensor([reward], device=device)

#         # Observe new state
#         last_screen = current_screen
        next_state = get_screen().unsqueeze(0)

        if not done:
          pass
            # diff = torch.sum(next_state - state).detach().cpu().numpy().item()
            # if diff == 0:
            #   reward = torch.tensor([-0.2], device=device)
        else:
            next_state = None
            # #todo: remove
            # break

#         # Store the transition in memory
        memory.push(state, action, next_state, reward)

#         # Move to the next state
        state = next_state

#         # Perform one step of the optimization (on the policy network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            #plot_durations()
            break

  # #todo: remove
  #   break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
        target_net.eval()

print('Complete')
# env.render()
env.close()
plt.ioff()
plt.show()

In [ ]:
len(episoe_states)

In [ ]:
env.max_steps = 500

In [ ]:
 EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)

In [ ]:
policy_net.eval()

In [ ]:
x  = policy_net(state)
torch.argmax(x), x, target_net(state)

In [ ]:
x = env.reset()

In [ ]:
state = get_screen().unsqueeze(0)
i2 = state.squeeze(0).squeeze(0) #.permute(1,2,0)
plt.gray()
plt.imshow(i2)
plt.show()

In [ ]:
action = select_action(state)
_, reward, done, _ = env.step(action.item())
action, reward

In [ ]:
next_state = get_screen()
i2 = next_state.squeeze(0).squeeze(0) 
plt.gray()
plt.imshow(i2)
plt.show()
state = next_state.unsqueeze(0)

In [ ]:
_, reward, done, _ = env.step(action.item())
reward

In [ ]:
x = policy_net(state)
x, torch.argmax(x)

In [ ]:
i2 = next_state.squeeze(0).squeeze(0) #.permute(1,2,0)
# plt.imshow(i2, cmap=plt.cm.gray)
plt.gray()
plt.imshow(i2)
plt.show()
state = next_state.unsqueeze(0)

In [ ]:

i2 = next_state.squeeze(0).permute(1,2,0)

plt.imshow(i2.numpy())

In [ ]:
state = next_state

In [ ]:
# BATCH_SIZE = 128
# GAMMA = 0.999
# EPS_START = 0.9
# EPS_END = 0.05
# EPS_DECAY = 10
# TARGET_UPDATE = 10
# import math 
# steps_done =1
# eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)

# eps_threshold

In [ ]:
(EPS_START - EPS_END) * math.exp(-1. * 0 / EPS_DECAY)

In [ ]:
(EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)

In [ ]:
eps_to_actions[3739][0:30]

In [ ]:
eps_to_actions[0][0:20]

In [ ]:
_, reward, done, _ = env.step(action.item())
reward

In [ ]:
torch.tensor([reward], device=device).detach().cpu().numpy().dtype